agent 5 Code share for data

In [ ]:
import collections
from queue import PriorityQueue
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from math import sqrt
from timeit import default_timer as timer
import random
import csv
import xlsxwriter
MovementDict = {'Grid':[],'Movement':[],'MinusInferance':[],'InferanceGrid':[]}

def PartialSensing(n, p):
    class Solver:
        def __init__(self):
            # a variable is a cell
            # constraints are all the equations that this cell is part of
            self.variables_to_constraints_map = {}

            # each constraint will a key value pair
            # key is a tuple, blocked_neighbours is a value
            # this is a list of constraints
            self.constraints = []

        def add_constraint(self, variable, n_list, value):
            variable_set = set()
            for i in n_list:
                #print('constraint')
                # print(i.x, i.y)
                variable_set.add((i.x * n + i.y))

            constraint = {}
            constraint[tuple(variable_set)] = value
            self.constraints.append(constraint)
            constraintIndex = len(self.constraints) - 1

            # adding constraint
            for v in variable_set:
                if self.variables_to_constraints_map.get(v) is None:
                    self.variables_to_constraints_map[v] = []
                self.variables_to_constraints_map[v].append(constraintIndex)

        def check_satisfaction_conditions(self, lhs, rhs):
            if rhs == 0:
                # all cells on lhs are free
                for i in range(0, len(lhs), 1):
                    cellX, cellY = divmod(lhs[i], n)
                    inferred_free(cellX, cellY)

            elif len(lhs) == rhs:
                # all cells on lhs are blocked
                for i in range(0, len(lhs), 1):
                    cellX, cellY = divmod(lhs[i], n)
                    inferred_blocked(cellX, cellY)

        def solve_constraint(self, cell, cell_value):
            # print("Solving constraint for ", cell.x, cell.y)
            # Solved when:
            # RHS becomes zero means all are free
            # Size of the list/set on left side = no. on the right side means all are blocked or all are free
            variable = cell.x * n + cell.y
            # get all constraint equations of this variable
            if self.variables_to_constraints_map.get(variable) is None:
                # print("No constraint exists for ", cell.x, cell.y)
                return

            constraints = self.variables_to_constraints_map[variable]
            for constraint_index in constraints:
                constraint = self.constraints[constraint_index]
                (lhs, rhs), = constraint.items()
                temp_lhs = list(lhs)
                temp_lhs.remove(variable)
                lhs = tuple(temp_lhs)

                if cell_value == 1:
                    rhs -= 1

                self.variables_to_constraints_map[variable].remove(constraint_index)
                constraint = {}
                constraint[lhs] = rhs
                self.constraints[constraint_index] = constraint
                self.check_satisfaction_conditions(lhs, rhs)
            return

    solver = Solver()

    def inferred_free(x, y):
        if knowledgeGrid.cellMap[(x, y)].confirmed:
            return
        knowledgeGrid.world[x, y] = 0
        knowledgeGrid.cellMap[(x, y)].confirmed = True
        # inform neighbours that this cell has been free
        informNeigboursAboutNewFreeCell(knowledgeGrid.cellMap[(x, y)])
        solver.solve_constraint(knowledgeGrid.cellMap[(x, y)], 0)

    def inferred_blocked(x, y):
        if knowledgeGrid.cellMap[(x, y)].confirmed:
            return

        knowledgeGrid.world[x, y] = 1
        knowledgeGrid.cellMap[(x, y)].confirmed = True
        # inform neighbours that this cell has been free
        informNeighboursAboutNewBlockedCell(knowledgeGrid.cellMap[(x, y)])
        solver.solve_constraint(knowledgeGrid.cellMap[(x, y)], 1)

    def agent_four_inference(cell, unknown_blocked):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.confirmed:
                if knowledgeGrid.world[i.x, i.y] == 1:
                    unknown_blocked -= 1
                # remove this from the equation because its value is known
                # if the value was 1, then that case was handled in the above line
                # if value was 0, then there is no effect on RHS
                neighbours.remove(i)

        solver.add_constraint(cell.x * n + cell.y, neighbours, unknown_blocked)
        return

    class Cell:
        # x = x-coordinate, y = y-coordinate in grid. Values from 0 to n-1
        def __init__(self, x, y, cost_from_start, total_cost, parent):
            self.x = x
            self.y = y
            self.cost_from_start = cost_from_start
            self.total_cost = total_cost
            self.total_neighbours = 0
            self.total_blocked_neighbours = 0
            self.known_neighbours = 0
            self.known_blocked_neighbours = 0
            self.parent = parent
            self.confirmed = False
            self.visited = False

        def __lt__(self, other):  # Added to add an object to the priority queue
            return self.total_cost < other.total_cost

    class Grid:
        def __init__(self, world, start, goal, path):
            self.world = world
            self.start = start
            self.goal = goal
            self.path = []
            self.cellMap = {}  # a dictionary to lookup a cell.

    def gridworld2(n, p):
        start = [0, 0];
        end = [n - 1, n - 1]
        gridworld = []

        for width in range(n):  # Loop creates n lists of n elements
            width = []
            for length in range(n):
                rand = np.random.random()
                if rand > p:
                    width.append(0)
                else:
                    width.append(1)
            gridworld.append(width)

        gridworld = np.array(gridworld)  # List of lists is turned into an array
        gridworld[start] = 0;
        gridworld[end] = 0
        return gridworld

    gridObj = Grid(gridworld2(n, p), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    knowledgeGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    KnowledgeGridMinusInferance = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) #declaration
    InferanceGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), []) #declaration
    def isCellWithinGrid(x, y, n):
        if (x < 0) or (x > (n - 1)) or (y < 0) or (y > (n - 1)):
            return False
        return True

    def isCellOpen(cell, n, grid):
        if isCellWithinGrid(cell.x, cell.y, n) == False:  # check if within gridworld bounds
            return False

        if grid.world[cell.x, cell.y] == 1:  # check if blocked
            return False

        return True

    def calculatefn(cell):
        goal = (n - 1, n - 1)
        estimate = calculateManhattanDistance(cell, goal)
        return estimate + cell.cost_from_start

    # Probability Start
    def calculateProbilityfn(cell):
        goal = (n - 1, n - 1)

        # when some neighbours are unknown
        if (cell.total_neighbours - cell.known_neighbours) != 0:
            estimateWithProbability = round(calculateManhattanDistance(cell, goal) * (((
                                                                                                   cell.total_blocked_neighbours - cell.known_blocked_neighbours) / (
                                                                                                   cell.total_neighbours - cell.known_neighbours)) + 1))

        # when all neighbours of a cell are known, no need to involve probabilistic calculations for them
        else:
            estimateWithProbability = calculateManhattanDistance(cell, goal)
        # print('Estimate : ' + str(estimateWithProbability))
        return estimateWithProbability + cell.cost_from_start
        # Probability End

    def calculateManhattanDistance(cell, goal):
        return abs(cell.x - goal[0]) + abs(cell.y - goal[1])

    def getNeighbours(cell):
        # east, south, west, north, ne, se, sw, nw in that order
        x = [1, 0, -1, 0, 1, 1, -1, -1]
        y = [0, 1, 0, -1, -1, 1, 1, -1]

        neighbours = []
        for i in range(8):
            if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                if knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i])) is None:
                    knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])] = Cell(cell.x + x[i], cell.y + y[i], 0, 0,
                                                                                 (cell.x, cell.y))
                neighbours.append(knowledgeGrid.cellMap[(cell.x + x[i], cell.y + y[i])])

        return neighbours

    def senseNeighbours(cell):
        ECell = Cell(cell.x + 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # East neighbour
        SCell = Cell(cell.x, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South neighbour
        WCell = Cell(cell.x - 1, cell.y, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # West neighbour
        NCell = Cell(cell.x, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North neighbour
        NECell = Cell(cell.x + 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North East neighbour
        SECell = Cell(cell.x + 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South East neighbour
        SWCell = Cell(cell.x - 1, cell.y + 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # South West neighbour
        NWCell = Cell(cell.x - 1, cell.y - 1, cell.cost_from_start + 1, 0, (cell.x, cell.y))  # North West neighbour

        potential_neighbours = [ECell, SCell, WCell, NCell, NECell, SECell, SWCell, NWCell]
        neighbours = [i for i in potential_neighbours if isCellWithinGrid(i.x, i.y, n)]
        total = 0
        blocked = 0
        for i in neighbours:
            total += 1
            if gridObj.world[i.x, i.y] == 1:
                blocked += 1
        cell.total_neighbours = total
        cell.total_blocked_neighbours = blocked
        return cell

    def getKnownNeighbours(cell):
        # east, south, west, north, ne, se, sw, nw in that order
        x = [1, 0, -1, 0, 1, 1, -1, -1]
        y = [0, 1, 0, -1, -1, 1, 1, -1]

        neighbours = []
        for i in range(8):
            if isCellWithinGrid(cell.x + x[i], cell.y + y[i], n):
                neighbourCell = knowledgeGrid.cellMap.get((cell.x + x[i], cell.y + y[i]))
                if neighbourCell is not None and neighbourCell.confirmed:
                    cell.known_neighbours += 1
                    if knowledgeGrid.world[cell.x + x[i], cell.y + y[i]] == 1:
                        cell.known_blocked_neighbours += 1
        return cell

    def updateKnownNeighbours(cell):
        knowledgeGrid.cellMap[(cell.x, cell.y)].known_neighbours += 1
        inferenceEngine(cell)

    def updateKnownBlockedNeighbours(cell):
        cell.known_blocked_neighbours += 1
        cell.known_neighbours += 1
        knowledgeGrid.cellMap[(cell.x, cell.y)] = cell
        inferenceEngine(cell)

    def inferenceEngine(cell):
        if cell.total_blocked_neighbours == cell.known_blocked_neighbours:
            markRemainingNeighboursAsFree(cell)

        # if totalFreeNeighbours = knownFreeNeighbours
        if (cell.total_neighbours - cell.total_blocked_neighbours
                == (cell.known_neighbours - cell.known_blocked_neighbours)):
            markRemainingNeighboursAsBlocked(cell)

        # Early Exit Function
        if cell.total_blocked_neighbours == cell.total_neighbours - 1:
            markRemainingNeighboursAsBlocked(cell)

    def markRemainingNeighboursAsFree(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            # this new freely marked cell must inform its neighbours which have been visited
            if i.confirmed == False:
                knowledgeGrid.world[i.x, i.y] = 0
                cell.known_neighbours += 1
                knowledgeGrid.cellMap[(i.x, i.y)] = cell
                i.confirmed = True
                knowledgeGrid.cellMap[(i.x, i.y)] = i
                # inform the neighbours
                informNeigboursAboutNewFreeCell(i)
                solver.solve_constraint(knowledgeGrid.cellMap[(i.x, i.y)], 0)

    def markRemainingNeighboursAsBlocked(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            # this new blocked cell must inform its neighbours which have been visited
            if i.confirmed == False:
                # print("Marking ", i.x, i.y, " as a blocked neighbour of ", cell.x, cell.y)
                cell.known_blocked_neighbours += 1
                cell.known_neighbours += 1
                knowledgeGrid.cellMap[(i.x, i.y)] = cell
                knowledgeGrid.world[i.x, i.y] = 1
                i.confirmed = True
                knowledgeGrid.cellMap[(i.x, i.y)] = i
                informNeighboursAboutNewBlockedCell(i)
                solver.solve_constraint(knowledgeGrid.cellMap[(i.x, i.y)], 1)

    def informNeigboursAboutNewFreeCell(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.visited:
                # print(cell.x, cell.y, " is a known free neighbour of ", i.x, i.y)
                updateKnownNeighbours(i)

    def informNeighboursAboutNewBlockedCell(cell):
        neighbours = getNeighbours(cell)
        for i in neighbours:
            if i.visited:
                # print(cell.x, cell.y, " is a known blocked neighbour of ", i.x, i.y)
                updateKnownBlockedNeighbours(i)
    #Ucode Start
    def IsGridSolvable(currentGrid, start):
        queue = PriorityQueue()
        closedMap = {}
        closedMap[(start.x, start.y)] = start
        start.parent = (start.x, start.y)
        queue.put((start.total_cost, start))  # first value in the tuple is priority(cost), second is the cell object

        goal = knowledgeGrid.goal

        while not queue.empty():  # while queue is not empty
            getter = queue.get()
            current = getter[1]

            rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                             (current.x, current.y))  # right neighbour
            downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # down neighbour
            leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # left neighbour
            topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                           (current.x, current.y))  # top neighbour

            neighbors = [rightCell, downCell, leftCell, topCell]  # calculate the f(n) for each neighbour of current

            for i in neighbors:
                i.total_cost = calculatefn(i)  # calculate the f(n) for each neighbour of current
                isItOpen = isCellOpen(i, n, currentGrid)
                if isCellOpen(i, n, currentGrid):
                    element = gridObj.cellMap.get((i.x, i.y))
                    if element is not None:
                        ''' 
                        If new path to neigbour costs less than previous path to it, then update cell.
                        This cell has already been visited before, but we are just checking
                        if the current path is less costlier than the one visited before
                        '''
                        if i.total_cost < gridObj.cellMap[(i.x, i.y)].total_cost:
                            gridObj.cellMap[(i.x, i.y)] = i

                    else:
                        gridObj.cellMap[(i.x, i.y)] = i  # this cell is being added first time
                        queue.put((i.total_cost, i))

            if current.x == goal.x and current.y == goal.y:  # To Do: Check if it's the goal and then update the goal if not
                return True
        return False

    def generateSolvableGrid():
        is_grid_solvable = False
        while not is_grid_solvable:
            gridObj = Grid(gridworld2(n, p), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
            # check if grid is solvable
            is_grid_solvable = IsGridSolvable(gridObj, Cell(0, 0, 0, (n * 2) - 2, (0, 0)))

        knowledgeGrid = Grid(gridworld2(n, 0), Cell(0, 0, 0, (n * 2) - 2, (0, 0)), Cell(n - 1, n - 1, 0, 0, (0, 0)), [])
    #Ucode end
    # this is the planning phase
    def AStar(knowledgeGrid, start):
        # print("Starting from ", start.x, start.y)
        queue = PriorityQueue()
        closedMap = {}
        closedMap[(start.x, start.y)] = start
        queue.put((start.total_cost, start))  # first value in the tuple is priority(cost), second is the cell object
        goal = knowledgeGrid.goal

        while not queue.empty():  # while queue is not empty
            getter = queue.get()
            current = getter[1]
            if isCellOpen(current, n, knowledgeGrid) is False and isCellWithinGrid(current.x, current.y, n) is True:
                knowledgeGrid.world[current.x, current.y] = 1

            if current.x == goal.x and current.y == goal.y:
                path = reconstructPath(current, start, closedMap)
                return executePlannedPath(path)

            rightCell = Cell(current.x + 1, current.y, current.cost_from_start + 1, 0,
                             (current.x, current.y))  # right neighbour
            downCell = Cell(current.x, current.y + 1, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # down neighbour
            leftCell = Cell(current.x - 1, current.y, current.cost_from_start + 1, 0,
                            (current.x, current.y))  # left neighbour
            topCell = Cell(current.x, current.y - 1, current.cost_from_start + 1, 0,
                           (current.x, current.y))  # top neighbour

            neighbors = [rightCell, downCell, leftCell, topCell]  # calculate the f(n) for each neighbour of current

            for i in neighbors:
                # i.total_cost = calculatefn(i)  # calculate the f(n) for each neighbour of current

                if knowledgeGrid.cellMap.get((i.x, i.y)) and knowledgeGrid.cellMap[(i.x, i.y)].visited:
                    i.total_cost = calculateProbilityfn(knowledgeGrid.cellMap[(i.x, i.y)])
                else:
                    i.total_cost = calculatefn(i)

                if isCellOpen(i, n, knowledgeGrid):
                    element = closedMap.get((i.x, i.y))
                    if element is not None:
                        ''' 
                        If new path to neigbour costs less than previous path to it, then update cell.
                        This cell has already been visited before, but we are just checking
                        if the current path is less costlier than the one visited before
                        '''
                        if i.total_cost < closedMap[(i.x, i.y)].total_cost:
                            closedMap[(i.x, i.y)] = i

                    else:
                        closedMap[(i.x, i.y)] = i  # this cell is being added first time
                        queue.put((i.total_cost, i))
            previous = current
        # queue is empty but goal node has not been reached
        return False

    def reconstructPath(current, start, closedMap):
        path = collections.deque()

        while True:
            if (current.x, current.y) == (start.x, start.y):
                break
            path.appendleft(current)
            current = closedMap[(current.parent[0], current.parent[1])]

        return path

    def directionArray(cell):
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (0, 1):
            return [1, 0, 0, 0]  # Right
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (0, -1):
            return [0, 1, 0, 0]  # Left
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (-1, 0):
            return [0, 0, 1, 0]  # Up
        if (cell.x - cell.parent[0], cell.y - cell.parent[1]) == (1, 0):
            return [0, 0, 0, 1]  # Down

    def executePlannedPath(path):
        grid = gridObj.world
        goal = gridObj.goal

        # print('total blocked at 00   ' + str(gridObj.cellMap[0, 0].total_blocked_neighbours))
        InferanceGrid.world[0, 0] = gridObj.cellMap[0, 0].total_blocked_neighbours

        for index, current in enumerate(path):
            # Ucode Start


            # print('actual values')
            # print(current.x, current.y)
            # print('Grid Minus Inferance:')
            #
            # print(KnowledgeGridMinusInferance.world)
            #MovementDict['InferanceGrid'].append(KnowledgeGridMinusInferance.world.flatten().tolist())
            # print('Inferance Values')
            # print(InferanceGrid.world.flatten().tolist())

            # Ucode End
            alreadyConfirmed = False
            if knowledgeGrid.cellMap.get((current.x, current.y)) is None:
                knowledgeGrid.cellMap[(current.x, current.y)] = current

            else:
                alreadyConfirmed = True
                current = knowledgeGrid.cellMap[(current.x, current.y)]

            if current.x == goal.x and current.y == goal.y:
                return knowledgeGrid.path
            #Ucode Start
            MovementDict['Grid'].append(knowledgeGrid.world.flatten().tolist())
            MovementDict['Movement'].append(directionArray(current))
            MovementDict['MinusInferance'].append(KnowledgeGridMinusInferance.world.flatten().tolist())
            MovementDict['InferanceGrid'].append(InferanceGrid.world.flatten().tolist())
            #Ucode End
            # hit blocked cell
            if grid[current.x, current.y] == 1:
                current.confirmed = True
                # print("Blocked by ", current.x, current.y)
                knowledgeGrid.world[current.x, current.y] = 1

                KnowledgeGridMinusInferance.world[current.x, current.y] = 1

                # setting restart point
                knowledgeGrid.start = gridObj.cellMap[current.parent[0], current.parent[1]]

                KnowledgeGridMinusInferance.world[current.parent[0], current.parent[1]]= 2

                # adding to the cell map
                knowledgeGrid.cellMap[(current.x, current.y)] = current

                # inform the neighbours that this is blocked
                if alreadyConfirmed == False:
                    informNeighboursAboutNewBlockedCell(current)

                solver.solve_constraint(current, 1)

                return None

            else:
                # can only sense neighbours of free cells
                current.confirmed = True
                current.visited = True
                current.total_neighbours = gridObj.cellMap[(current.x, current.y)].total_neighbours
                current.total_blocked_neighbours = gridObj.cellMap[(current.x, current.y)].total_blocked_neighbours
                InferanceGrid.world[(current.x, current.y)] = gridObj.cellMap[(current.x, current.y)].total_blocked_neighbours
                if alreadyConfirmed == False:
                    current = getKnownNeighbours(current)

                knowledgeGrid.cellMap[(current.x, current.y)] = current
                knowledgeGrid.path.append(current)
                inferenceEngine(current)

                # inform the neighbours that this is free
                if alreadyConfirmed == False:
                    informNeigboursAboutNewFreeCell(current)

                # advanced inference-
                agent_four_inference(current, current.total_blocked_neighbours)
                solver.solve_constraint(current, 0)


    def countBlockedNeighboursInRealGrid():
        # populate the information about blocked neighbours in the real grid
        for i in range(n):
            for j in range(n):
                cell = Cell(i, j, 0, 0, (0, 0))
                cell = senseNeighbours(cell)

                # create object for all cells in the grid
                gridObj.cellMap[(cell.x, cell.y)] = cell

    def repeatedAStar():
        generateSolvableGrid()
        countBlockedNeighboursInRealGrid()

        # mark start as confirmed
        knowledgeGrid.start.confirmed = True
        knowledgeGrid.start.total_blocked_neighbours = gridObj.cellMap[(0, 0)].total_blocked_neighbours
        knowledgeGrid.cellMap[(0, 0)] = knowledgeGrid.start

        # print(gridObj.world)
        # print()
        agent_four_inference(knowledgeGrid.start, knowledgeGrid.start.total_blocked_neighbours)

        while True:
            if len(knowledgeGrid.path) > 0:
                previousStartPoint = knowledgeGrid.path[len(knowledgeGrid.path) - 1]

                # if we are restarting from the same point where we restarted previously, then dont add this to the path again
                if knowledgeGrid.start.x != previousStartPoint.x and knowledgeGrid.start.y != previousStartPoint.y:
                    knowledgeGrid.path.append(knowledgeGrid.start)

            path = AStar(knowledgeGrid, knowledgeGrid.start)
            if path is None:
                continue

            elif path == False:
                # print("Knowledge grid line 279:")
                # print(knowledgeGrid.world)
                return False

            else:
                knowledgeGrid.world[0, 0] = 5
                knowledgeGrid.world[knowledgeGrid.start.x, knowledgeGrid.start.y] = 5

                for i in knowledgeGrid.path:
                    # print('path')
                    # print(i.x, i.y)
                    knowledgeGrid.world[i.x, i.y] = 5

                knowledgeGrid.world[knowledgeGrid.goal.x, knowledgeGrid.goal.y] = 5
                # print("Knowledge grid line 291:")
                # print(knowledgeGrid.world)
                return True
    return repeatedAStar()


for i in range(20):
    PartialSensing(101, 0.3)

import pandas as pd
# df = pd.DataFrame(MovementDict)
df = pd.DataFrame.from_dict(MovementDict, orient='index')
df = df.transpose()
df.to_excel('Project 2 data file 1.xlsx', index=False)
print('completed')